In [1]:
import cv2
import numpy as np
import tensorflow as tf

In [13]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(1932.4979, shape=(), dtype=float32)


2024-03-22 15:11:06.964399: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 15:11:06.967294: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 3393620000 Hz
2024-03-22 15:11:06.967420: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1536850 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-03-22 15:11:06.967427: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [19]:
class CameraObjectDetector:
    def __init__(self, model_path):
        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        _, self.input_height, self.input_width, _ = self.input_details[0]['shape']

detector = CameraObjectDetector('models/efficientNetLite2.tflite')


AttributeError: 'CameraObjectDetector' object has no attribute 'run'

In [5]:
def preprocess(self, frame):
        frame_resized = cv2.resize(frame, (self.input_width, self.input_height))
        input_data = np.expand_dims(frame_resized, axis=0).astype(np.float32)
        input_mean, input_std = 127.5, 127.5
        input_data = (input_data - input_mean) / input_std
        return input_data

In [6]:
def postprocess(self, frame, boxes, classes, scores, count):
        for i in range(count):
            if scores[i] > 0.5:  # Only consider detections with a confidence > 50%
                ymin, xmin, ymax, xmax = boxes[i]
                xmin = int(xmin * self.input_width)
                xmax = int(xmax * self.input_width)
                ymin = int(ymin * self.input_height)
                ymax = int(ymax * self.input_height)

                # Draw the bounding box and label on the frame
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                label = f"{classes[i]}: {int(scores[i]*100)}%"
                cv2.putText(frame, label, (xmin, ymax), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
        return frame

In [7]:
def detect_objects(self, image):
        self.interpreter.set_tensor(self.input_details[0]['index'], image)
        self.interpreter.invoke()
        boxes = self.interpreter.get_tensor(self.output_details[0]['index'])[0]
        classes = self.interpreter.get_tensor(self.output_details[1]['index'])[0]
        scores = self.interpreter.get_tensor(self.output_details[2]['index'])[0]
        count = self.interpreter.get_tensor(self.output_details[3]['index'])[0]
        return boxes, classes, scores, count


In [8]:
def run(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Cannot open camera")
            exit()
        print("Camera opened successfully.")
        frame_count = 0  # Counter for the number of frames processed
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Can't receive frame (stream end?). Exiting ...")
                break
        
            # Process frame for object detection.
            input_data = self.preprocess(frame)
            boxes, classes, scores, count = self.detect_objects(input_data)
            frame = self.postprocess(frame, boxes, classes, scores, count)

            # Increment and print the frame count every 100 frames.
            frame_count += 1
            if frame_count % 100 == 0:
                print(f"Processed {frame_count} frames")

            # Display the resulting frame with bounding boxes
            cv2.imshow('Object Detection', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) == ord('q'):
                print("Quitting the application...")
            break

        cap.release()
        cv2.destroyAllWindows()
        print("Camera feed has been released and windows are closed.")
    
    detector.run()